In [1]:
#https://keras.io/examples/rl/ppo_cartpole/
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Multiply
import scipy.signal
import time
import pandas as pd
import random
import string
import random

### Environment Functions

In [2]:
# https://gist.github.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b
my_file = open("C:/Users/rudra/OneDrive/Desktop/reinforcementLearning/Wordle/data/wordle-answers-alphabetical.txt", "r")
content = my_file.read()
content = list(content.split('\n'))
content = random.sample(content, 100)

In [4]:
lower_alphabet = list(string.ascii_letters)[:26]

In [5]:
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
observation_dimensions = 182
num_actions = len(content)


In [6]:
def get_secret_word():
    return random.choice(content)

In [7]:
def reset_available_action_space():
    return [1.0 for i in range(len(content))]

def reset_guessed_alphabet_state():
    return [0 for i in range(len(lower_alphabet))]

# array of 26 which represents which alphabet is available in word
def reset_contains_alphabet_state():
    return [0 for i in range(len(lower_alphabet))]

# Array of 26*5. 
# First 26 represent which alphabet was correctly guessed at the first slot
# Second 26 represent which alphabet was correctly guessed at the second slot. And so on for the next 5 slots.
def reset_correct_alphabet_pos_state():
    return [0 for i in range(len(lower_alphabet)*5)]

In [8]:
def update_AVAILABLE_ACTION_SPACE(action):
    AVAILABLE_ACTION_SPACE[action] = 0

In [9]:
def env_reset():
    AVAILABLE_ACTION_SPACE = reset_available_action_space()
    guessed_alphabet_state = reset_guessed_alphabet_state()
    contains_alphabet_state = reset_contains_alphabet_state()
    correct_alphabet_pos_state = reset_correct_alphabet_pos_state()
    state = np.array(guessed_alphabet_state + contains_alphabet_state + correct_alphabet_pos_state)
    SECRET_WORD = get_secret_word()
    return state, SECRET_WORD, AVAILABLE_ACTION_SPACE

In [10]:
def env_step(action, observation):
    state = observation[0].tolist()
    guessed_word = content[action]
    guessed_alphabet_state = state[:26]
    contains_alphabet_state = state[26:52]
    correct_alphabet_pos_state = state[52:]
    
    done = False
    reward = 0
    
    if SECRET_WORD == guessed_word:
        done = True
        reward = 10
    secret_word = list(SECRET_WORD)
    guessed_word = list(guessed_word)
    for index_, char_ in enumerate(guessed_word):
        alphabet_index = lower_alphabet.index(char_)
        guessed_alphabet_state[alphabet_index] = 1
        if char_ in secret_word:
            contains_alphabet_state[alphabet_index] = 1
            if secret_word[index_] == char_:
                correct_alphabet_pos_state[26*index_ + alphabet_index] = 1
    state = np.array(guessed_alphabet_state + contains_alphabet_state + correct_alphabet_pos_state)
    return state, reward, done

### PPO

In [11]:
def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]


class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size, observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )


def mlp(x, sizes, activation=tf.tanh, output_activation=None):
    # Build a feedforward neural network
    for size in sizes[:-1]:
        x = layers.Dense(units=size, activation=activation)(x)
    return layers.Dense(units=sizes[-1], activation=output_activation)(x)


def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = tf.nn.log_softmax(logits)
    logprobability = tf.reduce_sum(
        tf.one_hot(a, num_actions) * logprobabilities_all, axis=1
    )
    return logprobability


# Sample action from actor
# @tf.function
def sample_action(observation):
    logits = actor(observation)
    available_action_tensor = tf.expand_dims(tf.convert_to_tensor(AVAILABLE_ACTION_SPACE), 0)
    logits = tf.add(logits, 1.0)
    logits = Multiply()([logits, available_action_tensor])
    logits = tf.add(logits, -1.0)
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
    return logits, action


# Train the policy by maxizing the PPO-Clip objective
@tf.function
def train_policy(
    observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
):

    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = tf.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = tf.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -tf.reduce_mean(
            tf.minimum(ratio * advantage_buffer, min_advantage)
        )
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = tf.reduce_mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = tf.reduce_sum(kl)
    return kl


# Train the value function by regression on mean-squared error
@tf.function
def train_value_function(observation_buffer, return_buffer):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = tf.reduce_mean((return_buffer - critic(observation_buffer)) ** 2)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))

In [12]:
# Hyperparameters of the PPO algorithm
games_per_epoch = 10
game_length = 6
epochs = 100000
gamma = 0.99
clip_ratio = 0.2
policy_learning_rate = 3e-4
value_function_learning_rate = 1e-3
train_policy_iterations = 80
train_value_iterations = 80
lam = 0.97
target_kl = 0.01
hidden_sizes = (256, 128)

In [13]:

# Initialize the buffer
buffer = Buffer(observation_dimensions, games_per_epoch*game_length)

# Initialize the actor and the critic as keras models
observation_input = keras.Input(shape=(observation_dimensions,), dtype=tf.float32)
logits = mlp(observation_input, list(hidden_sizes) + [num_actions], tf.tanh, None)
actor = keras.Model(inputs=observation_input, outputs=logits)
value = tf.squeeze(
    mlp(observation_input, list(hidden_sizes) + [1], tf.tanh, None), axis=1
)
critic = keras.Model(inputs=observation_input, outputs=value)

# Initialize the policy and the value function optimizers
policy_optimizer = keras.optimizers.Adam(learning_rate=policy_learning_rate)
value_optimizer = keras.optimizers.Adam(learning_rate=value_function_learning_rate)

# Initialize the observation, episode return and episode length
observation, SECRET_WORD, AVAILABLE_ACTION_SPACE = env_reset()
episode_return, episode_length = 0, 0

In [ ]:
# Iterate over the number of epochs
for epoch in range(epochs):
    # Initialize the sum of the returns, lengths and number of episodes for each epoch
    sum_return = 0
    sum_length = 0
    num_episodes = 0

    # Iterate over the steps of each epoch
    for game_in_epoch in range(games_per_epoch):
        for game in range(game_length):
        # Get the logits, action, and take one step in the environment
            observation = observation.reshape(1, -1)
            logits, action = sample_action(observation)
            action_index = int(action[0].numpy())
            update_AVAILABLE_ACTION_SPACE(action_index)
            observation_new, reward, done = env_step(action_index, observation)
            reward = reward * (game_length - game)
            episode_return += reward
            episode_length += 1

            # Get the value and log-probability of the action
            value_t = critic(observation)
            logprobability_t = logprobabilities(logits, action)

            # Store obs, act, rew, v_t, logp_pi_t
            buffer.store(observation, action, reward, value_t, logprobability_t)

            # Update the observation
            observation = observation_new

            # Finish trajectory if reached to a terminal state
            terminal = done
            if terminal or game == game_length-1:
                last_value = 0 if done else critic(observation.reshape(1, -1))
                buffer.finish_trajectory(last_value)
                sum_return += episode_return
                sum_length += episode_length
                num_episodes += 1
                observation, SECRET_WORD, AVAILABLE_ACTION_SPACE = env_reset()
                episode_return, episode_length = 0, 0
                break

    # Get values from the buffer
    (
        observation_buffer,
        action_buffer,
        advantage_buffer,
        return_buffer,
        logprobability_buffer,
    ) = buffer.get()

    # Update the policy and implement early stopping using KL divergence
    for _ in range(train_policy_iterations):
        kl = train_policy(
            observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
        )
        if kl > 1.5 * target_kl:
            # Early Stopping
            break

    # Update the value function
    for _ in range(train_value_iterations):
        train_value_function(observation_buffer, return_buffer)

    # Print mean return and length for each epoch
    if epoch % 10 == 0:
        print(
            f" Epoch: {epoch + 1}. Mean Return: {sum_return / num_episodes}. Mean Length: {sum_length / num_episodes}"
        )                

 Epoch: 1. Mean Return: 6.0. Mean Length: 5.5
 Epoch: 11. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 21. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 31. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 41. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 51. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 61. Mean Return: 6.0. Mean Length: 5.5
 Epoch: 71. Mean Return: 13.0. Mean Length: 5.0
 Epoch: 81. Mean Return: 2.0. Mean Length: 5.9
 Epoch: 91. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 101. Mean Return: 4.0. Mean Length: 5.7
 Epoch: 111. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 121. Mean Return: 10.0. Mean Length: 5.2
 Epoch: 131. Mean Return: 5.0. Mean Length: 5.7
 Epoch: 141. Mean Return: 7.0. Mean Length: 5.5
 Epoch: 151. Mean Return: 3.0. Mean Length: 5.8
 Epoch: 161. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 171. Mean Return: 5.0. Mean Length: 5.6
 Epoch: 181. Mean Return: 5.0. Mean Length: 5.6
 Epoch: 191. Mean Return: 0.0. Mean Length: 6.0
 Epoch: 201. Mean Return: 0.0. Mean Length: 6.0
 